In [1]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate

In [2]:
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain.storage import InMemoryStore
from langchain_ollama import OllamaEmbeddings
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [3]:
embeddings = OllamaEmbeddings(model='llama3')
vectorstore = Chroma(persist_directory='../data/chroma_db', embedding_function=embeddings)

store = InMemoryStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [4]:
from langchain_core.runnables import RunnablePassthrough

model = ChatOllama(model="llama2:13b-chat", temperature=0)

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [5]:
# Testing the rag chain

chain.invoke(
       "Exaplin the approach of the paper?"
)

ResponseError: model "llama3" not found, try pulling it first